# core

> Audio Embeddings

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from fastcore.all import *

## AudioArray

`AudioArray` is the main audio representation in `auditus`. It stores audio data as a NumPy array and the sample rate. It provides utilities for displaying and parsing audio.

In [ ]:
#| export
class AudioArray(BasicRepr): 
    def __init__(self, a: np.ndarray, sr: int): store_attr()
    @property
    def shape(self): return self.a.shape
    def audio(self): 
        from IPython.display import Audio
        return Audio(self.a, rate=self.sr)
    def __len__(self): return len(self.a)
    def __getitem__(self, idx): return self.a[idx]

Here is an example of a pink noise signal.

In [ ]:
aa = AudioArray(np.cumsum(np.random.normal(0, 0.01, 32_000*10)), 32_000)
aa

AudioArray(a=array([-0.0044617 ,  0.01129441,  0.02002544, ...,  2.30104054,
        2.30193327,  2.31287792]), sr=32000)

In [ ]:
aa.audio()

In [ ]:
aa[:5], aa.sr

(array([-0.0044617 ,  0.01129441,  0.02002544,  0.02193562,  0.01722719]),
 32000)

In [ ]:
test_eq(len(aa), 320000)
test_eq(aa.sr, 32_000)
test_eq(aa.shape, (320000,))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()